## Perform CRUD operations using Crunchy PostgreSQL for Kubernetes Operator 

#### 1. Install required libraries

In [ ]:
pip install psycopg2-binary

In [ ]:
import psycopg2
import psycopg2.errorcodes
import time
import logging
import random

#### 2. Connect to Crunchy PostgreSQL for Kubernetes Operator 

In [ ]:
conn = psycopg2.connect(database="hippo", user="hippo", password="datalake", host="127.0.0.1", port="5432")

#### 3. Create a table and populate the data

In [ ]:
def create_accounts(conn):
    with conn.cursor() as cur:
        cur.execute('CREATE TABLE IF NOT EXISTS accounts (id INT PRIMARY KEY, balance INT)')
        cur.execute('INSERT INTO accounts (id, balance) VALUES (1, 1000), (2, 250)')
        logging.debug("create_accounts(): status message: {}".format(cur.statusmessage))
    conn.commit()

In [ ]:
create_accounts(conn)

#### 4. Read/Query the data from the table

In [ ]:
def query(conn):
    with conn.cursor() as cur:
        cur.execute("SELECT id, balance FROM accounts")
        logging.debug("print_balances(): status message: {}".format(cur.statusmessage))
        rows = cur.fetchall()
        conn.commit()
        print("Balances at {}".format(time.asctime()))
        for row in rows:
            print([str(cell) for cell in row])

In [ ]:
query(conn)

#### 5. Perform required operations and Update the data in the table

In [ ]:
def run_transaction(conn, op):
    retries = 0
    max_retries = 3
    with conn:
        while True:
            retries +=1
            if retries == max_retries:
                err_msg = "Transaction did not succeed after {} retries".format(max_retries)
                raise ValueError(err_msg)

            try:
                op(conn)

                # If we reach this point, we were able to commit, so we break
                # from the retry loop.
                break
            except psycopg2.Error as e:
                logging.debug("e.pgcode: {}".format(e.pgcode))
                if e.pgcode == '40001':
                    # This is a retry error, so we roll back the current
                    # transaction and sleep for a bit before retrying. The
                    # sleep time increases for each failed transaction.
                    conn.rollback()
                    logging.debug("EXECUTE SERIALIZATION_FAILURE BRANCH")
                    sleep_ms = (2**retries) * 0.1 * (random.random() + 0.5)
                    logging.debug("Sleeping {} seconds".format(sleep_ms))
                    time.sleep(sleep_ms)
                    continue
                else:
                    logging.debug("EXECUTE NON-SERIALIZATION_FAILURE BRANCH")
                    raise e

In [ ]:
def transfer_funds(conn, frm, to, amount):
    with conn.cursor() as cur:

        # Check the current balance.
        cur.execute("SELECT balance FROM accounts WHERE id = " + str(frm))
        from_balance = cur.fetchone()[0]
        if from_balance < amount:
            err_msg = "Insufficient funds in account {}: have {}, need {}".format(frm, from_balance, amount)
            raise RuntimeError(err_msg)

        # Perform the transfer.
        cur.execute("UPDATE accounts SET balance = balance - %s WHERE id = %s",
                    (amount, frm))
        cur.execute("UPDATE accounts SET balance = balance + %s WHERE id = %s",
                    (amount, to))
    conn.commit()
    logging.debug("transfer_funds(): status message: {}".format(cur.statusmessage))

In [ ]:
amount = 100
fromId = 1
toId = 2

try:
    run_transaction(conn, lambda conn: transfer_funds(conn, fromId, toId, amount))
except ValueError as ve:
    logging.debug("run_transaction(conn, op) failed: {}".format(ve))
    pass

In [ ]:
query(conn)

#### 6. Delete the data content

In [ ]:
def delete_accounts(conn):
    with conn.cursor() as cur:
        cur.execute("DELETE FROM hippo.accounts WHERE id=2")
        logging.debug("delete_accounts(): status message: {}".format(cur.statusmessage))
    conn.commit()

In [ ]:
delete_accounts(conn)

In [ ]:
query(conn)

#### 7. Drop the table

In [ ]:
def delete_table(conn):
    with conn.cursor() as cur:
        cur.execute("DROP TABLE hippo.accounts")
        logging.debug("delete_accounts(): status message: {}".format(cur.statusmessage))
    conn.commit()

#### Uncomment the below step to delete the table

In [ ]:
#delete_table(conn)

#### 8. Close communication with the database.

In [ ]:
conn.close()